In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

import scipy.sparse as sparse
import random
import implicit
from implicit.als import AlternatingLeastSquares as ALS

%cd /home/user_3/medistream-recsys/Script
from preprocessing import drop_columns,dict_to_column,dict_to_set,set_to_column,key_to_element

pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', 100)

/home/user_3/medistream-recsys/Script


In [2]:
df = pd.read_json('/fastcampus-data/select_column_version_4.json')
df

,_id,date_created,regular_price,sale_price,three_months,date_paid,customer_id,paid,name_x,category_id_y,product_ids,quantity,price,price_total,age_group,한의사 여부,사업자 여부,cancelled,name,slug
0,5d60cebb4e77525ec5ca141c,None,NaN,NaN,NaN,None,5d60cea34e77525ec5ca1413,False,None,None,None,NaN,NaN,NaN,20-29,1.0,0.0,False,None,None
1,5d60daf14e77525ec5ca1497,2019-08-21T05:04:55.728Z,66000.0,0.0,NaN,None,5d60ceba4e77525ec5ca1419,False,제일 경옥고,5cf8bbba0098b2225c5dfaa2,5d5cd0f74e77525ec5ca12ab,3.0,59400.0,178200.0,20-29,1.0,0.0,True,의약품,medicine
2,5d60d2d04e77525ec5ca145b,2019-08-21T05:04:55.728Z,66000.0,0.0,NaN,2019-09-06T09:13:26.749Z,5d60d0a94e77525ec5ca1446,True,제일 경옥고,5cf8bbba0098b2225c5dfaa2,5d5cd0f74e77525ec5ca12ab,1.0,59400.0,59400.0,30-39,0.0,1.0,False,의약품,medicine
3,5d60e27c4e77525ec5ca14b6,2019-10-16T04:03:00.777Z,44800.0,37900.0,0.0,2019-10-21T02:18:23.528Z,5d60e2694e77525ec5ca14b5,True,"한의원 원장님도 기획자가 돼야한다 생각한다면, 읽어야 하는 책",5cf8bbba0098b2225c5dfaa3,5da696740dabe405b156eb5c,1.0,37000.0,37000.0,20-29,0.0,0.0,False,도서,book
4,5d60f8764e77525ec5ca14e5,None,NaN,NaN,NaN,None,5d60f3a74e77525ec5ca14da,False,None,None,None,NaN,NaN,NaN,40-49,1.0,1.0,False,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
266653,632044cb6c2e37002369f129,2022-02-16T04:01:35.033Z,41500.0,35200.0,NaN,2022-09-13T08:53:42.268Z,5d918f180dabe405b156d5e5,True,"[씨케이주식회사] 황기(편) (5묶음) 600g*5ea, 중국",5fb32f7a5120316334323fd8,620c771fd41623002392d61b,1.0,35200.0,35200.0,30-39,0.0,1.0,False,한약재,medicinal-herbs
266654,632044cb6c2e37002369f129,2020-11-20T06:59:35.395Z,4900.0,4200.0,NaN,2022-09-13T08:53:42.268Z,5d918f180dabe405b156d5e5,True,"[씨케이주식회사] 어성초 600g, 중국",5fb32f7a5120316334323fd8,5fb769575120316334324a69,1.0,4200.0,4200.0,30-39,0.0,1.0,False,한약재,medicinal-herbs
266655,632044cb6c2e37002369f129,2020-11-20T06:59:36.047Z,3900.0,3300.0,NaN,2022-09-13T08:53:42.268Z,5d918f180dabe405b156d5e5,True,"[씨케이주식회사] 여정실 600g, 중국",5fb32f7a5120316334323fd8,5fb769585120316334324a6a,1.0,3300.0,3300.0,30-39,0.0,1.0,False,한약재,medicinal-herbs
266656,632045696c2e37002369f136,2022-08-30T00:42:13.239Z,330000.0,0.0,386.0,2022-09-13T08:59:21.151Z,628ecb419d9388002406f007,True,맘 편하게 한약 처방하기,5ea40b671a3ca36b967849ad,630d5ce568554000236d9a5d,1.0,264000.0,264000.0,None,1.0,0.0,False,온라인 강의,lecture


In [7]:
date_state = "2022-05-12"

# paid orders만 가져오기
df['date_paid'] = pd.to_datetime(df['date_paid'])
df_only_paid = df[~df['date_paid'].isna()]
# 3개월치 데이터만 가져오기
df_date = df_only_paid[df_only_paid['date_paid'] > date_state]
# 취소 안된 것만 가져오기
complete_df = df_date[(df_date['paid'] == True) & (df_date['cancelled']==False)]
# # 도서 카테고리만 가져오기
# only_book = complete_df[complete_df['name'] == '도서']

# 유저가 중복으로 아이템 구매 삭제
df_duplicated_book = complete_df.drop_duplicates(subset=['customer_id','product_ids'])
df_all = df_duplicated_book.sort_values(by='date_paid').reset_index(drop=True)


In [8]:
df_all['name'].value_counts()

온라인 강의     30994
도서          5910
소모품         3422
한약재         3175
의료기기        1050
식품           957
의약품          890
메디TV         840
치료실 물품       718
의류/잡화        666
생활           582
오프라인 강의      292
가전/가구        234
디자인          122
구독            95
해부학 모형        64
재활 용품         47
약속처방          37
기타            27
기획서비스          3
Name: name, dtype: int64

In [11]:
df_all[['product_ids','name_x']].value_counts()

product_ids               name_x                         
62821ca79d93880024064505  [제32기 전졸협] 국시특강 간계내과학              828
62821e079d93880024064575  [제32기 전졸협] 국시특강 신경정신과학             828
62821d249d93880024064523  [제32기 전졸협] 국시특강 폐계내과학              828
62821d459d9388002406452b  [제32기 전졸협] 국시특강 신계내과학              828
62821d649d93880024064536  [제32기 전졸협] 국시특강 상한론                828
                                                            ... 
5fa8e38751203163343224c4  일차진료아카데미 처방가이드                       1
5f167b2f221c8b5f77c8f6a4  신경블록 술기 아틀라스                         1
609a3735c1908b001a6418f8  [신흥제약] 황기밀자 중국 500g, 중국              1
62905a5e9d9388002406f83c  아스페카 스트레이트 스크럽 여성 팬츠 Royal Blue      1
60dbdd89d26af90681b1a3d5  에이치알메디텍 미용기 스킨프로                     1
Length: 2029, dtype: int64

In [12]:
df_all['date_paid'] = df_all['date_paid'].dt.strftime('%m/%d/%Y')

In [16]:
df_all.pivot_table('quantity','name_x','date_paid',aggfunc='sum',margins=True).sort_values(by='All',ascending=False).head(10)

date_paid,05/12/2022,05/13/2022,05/14/2022,05/15/2022,05/16/2022,05/17/2022,05/18/2022,05/19/2022,05/20/2022,05/21/2022,05/22/2022,05/23/2022,05/24/2022,05/25/2022,05/26/2022,05/27/2022,05/28/2022,05/29/2022,05/30/2022,05/31/2022,06/01/2022,06/02/2022,06/03/2022,06/04/2022,06/05/2022,06/06/2022,06/07/2022,06/08/2022,06/09/2022,06/10/2022,06/11/2022,06/12/2022,06/13/2022,06/14/2022,06/15/2022,06/16/2022,06/17/2022,06/18/2022,06/19/2022,06/20/2022,06/21/2022,06/22/2022,06/23/2022,06/24/2022,06/25/2022,06/26/2022,06/27/2022,06/28/2022,06/29/2022,06/30/2022,...,07/27/2022,07/28/2022,07/29/2022,07/30/2022,07/31/2022,08/01/2022,08/02/2022,08/03/2022,08/04/2022,08/05/2022,08/06/2022,08/07/2022,08/08/2022,08/09/2022,08/10/2022,08/11/2022,08/12/2022,08/13/2022,08/14/2022,08/15/2022,08/16/2022,08/17/2022,08/18/2022,08/19/2022,08/20/2022,08/21/2022,08/22/2022,08/23/2022,08/24/2022,08/25/2022,08/26/2022,08/27/2022,08/28/2022,08/29/2022,08/30/2022,08/31/2022,09/01/2022,09/02/2022,09/03/2022,09/04/2022,09/05/2022,09/06/2022,09/07/2022,09/08/2022,09/09/2022,09/10/2022,09/11/2022,09/12/2022,09/13/2022,All
name_x,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
All,415.0,269.0,182.0,137.0,508.0,400.0,243.0,278.0,14212.0,2487.0,61.0,436.0,454.0,403.0,422.0,343.0,131.0,96.0,737.0,698.0,334.0,311.0,349.0,107.0,71.0,89.0,360.0,3786.0,323.0,407.0,91.0,90.0,479.0,517.0,214.0,341.0,361.0,118.0,94.0,454.0,378.0,286.0,438.0,324.0,91.0,135.0,547.0,455.0,655.0,3155.0,...,298.0,491.0,381.0,83.0,137.0,512.0,398.0,233.0,295.0,343.0,98.0,110.0,405.0,382.0,400.0,475.0,1786.0,62.0,35.0,105.0,431.0,249.0,223.0,258.0,79.0,67.0,295.0,526.0,292.0,203.0,296.0,143.0,123.0,319.0,683.0,483.0,493.0,448.0,94.0,173.0,277.0,282.0,195.0,286.0,42.0,21.0,50.0,139.0,361.0,63942.0
[제32기 전졸협] 국시특강 상한론,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,685.0,134.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,828.0
[제32기 전졸협] 국시특강 폐계내과학,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,685.0,134.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,828.0
[제32기 전졸협] 국시특강 침구학,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,685.0,134.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,828.0
[제32기 전졸협] 국시특강 외과학,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,685.0,134.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,828.0
[제32기 전졸협] 국시특강 예방의학,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,685.0,134.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,828.0
[제32기 전졸협] 국시특강 안이비인후과학,NaN,NaN,Na

In [19]:
df_consum = df_all[df_all['name'] == '소모품']
consum_pivot = df_consum.pivot_table('quantity','name_x','date_paid',aggfunc='sum',margins=True).sort_values(by='All',ascending=False)
consum_pivot

date_paid,05/12/2022,05/13/2022,05/14/2022,05/15/2022,05/16/2022,05/17/2022,05/18/2022,05/19/2022,05/20/2022,05/21/2022,05/22/2022,05/23/2022,05/24/2022,05/25/2022,05/26/2022,05/27/2022,05/28/2022,05/29/2022,05/30/2022,05/31/2022,06/01/2022,06/02/2022,06/03/2022,06/04/2022,06/05/2022,06/06/2022,06/07/2022,06/08/2022,06/09/2022,06/10/2022,06/11/2022,06/12/2022,06/13/2022,06/14/2022,06/15/2022,06/16/2022,06/17/2022,06/18/2022,06/19/2022,06/20/2022,06/21/2022,06/22/2022,06/23/2022,06/24/2022,06/25/2022,06/26/2022,06/27/2022,06/28/2022,06/29/2022,06/30/2022,...,07/27/2022,07/28/2022,07/29/2022,07/30/2022,07/31/2022,08/01/2022,08/02/2022,08/03/2022,08/04/2022,08/05/2022,08/06/2022,08/07/2022,08/08/2022,08/09/2022,08/10/2022,08/11/2022,08/12/2022,08/13/2022,08/14/2022,08/15/2022,08/16/2022,08/17/2022,08/18/2022,08/19/2022,08/20/2022,08/21/2022,08/22/2022,08/23/2022,08/24/2022,08/25/2022,08/26/2022,08/27/2022,08/28/2022,08/29/2022,08/30/2022,08/31/2022,09/01/2022,09/02/2022,09/03/2022,09/04/2022,09/05/2022,09/06/2022,09/07/2022,09/08/2022,09/09/2022,09/10/2022,09/11/2022,09/12/2022,09/13/2022,All
name_x,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
All,45.0,46.0,35.0,7.0,195.0,176.0,82.0,69.0,70.0,19.0,5.0,124.0,131.0,105.0,78.0,130.0,63.0,47.0,154.0,288.0,155.0,63.0,91.0,46.0,40.0,49.0,152.0,145.0,96.0,101.0,38.0,25.0,86.0,140.0,72.0,141.0,91.0,7.0,25.0,116.0,125.0,109.0,57.0,136.0,26.0,56.0,137.0,156.0,121.0,86.0,...,110.0,78.0,215.0,28.0,49.0,166.0,97.0,45.0,91.0,107.0,24.0,15.0,92.0,153.0,53.0,19.0,14.0,15.0,1.0,21.0,134.0,38.0,42.0,78.0,17.0,14.0,142.0,145.0,100.0,53.0,49.0,52.0,24.0,148.0,123.0,135.0,63.0,54.0,7.0,49.0,79.0,53.0,44.0,22.0,8.0,6.0,1.0,28.0,89.0,9582.0
세이프란 전용 란셋 니들 30g (100pcs) 1box,NaN,NaN,26.0,NaN,10.0,6.0,NaN,26.0,6.0,10.0,NaN,31.0,50.0,8.0,15.0,16.0,7.0,10.0,10.0,36.0,12.0,NaN,1.0,6.0,NaN,NaN,40.0,NaN,10.0,20.0,NaN,10.0,NaN,25.0,10.0,12.0,NaN,NaN,NaN,6.0,10.0,NaN,NaN,10.0,NaN,NaN,10.0,25.0,NaN,NaN,...,NaN,4.0,NaN,NaN,NaN,2.0,NaN,NaN,30.0,10.0,NaN,NaN,NaN,30.0,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,3.0,3.0,NaN,NaN,10.0,23.0,9.0,10.0,NaN,26.0,NaN,9.0,10.0,2.0,NaN,2.0,NaN,NaN,NaN,6.0,10.0,3.0,NaN,NaN,NaN,NaN,14.0,804.0
성심 인슐린주사기 1mL/cc 100pcs,NaN,18.0,NaN,NaN,10.0,45.0,5.0,6.0,15.0,NaN,NaN,5.0,NaN,20.0,5.0,NaN,NaN,NaN,3.0,16.0,10.0,16.0,10.0,NaN,10.0,NaN,10.0,5.0,6.0,5.0,NaN,NaN,17.0,10.0,NaN,8.0,NaN,NaN,NaN,22.0,NaN,NaN,NaN,12.0,15.0,NaN,10.0,11.0,17.0,15.0,...,NaN,NaN,12.0,NaN,NaN,5.0,5.0,NaN,NaN,10.0,6.0,NaN,10.0,46.0,NaN,NaN,NaN,NaN,NaN,NaN,5.0,5.0,NaN,6.0,NaN,NaN,20.0,NaN,6.0,10.0,10.0,NaN,NaN,17.0,8.0,10.0,1.0,1.0,NaN,5.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,713.0
성심 일회용 주사침 (기본사이즈) 100pcs,NaN,2.0,NaN,NaN,34.0,6.0,NaN,NaN,NaN,NaN,NaN,10.0,5.0,1.0,NaN,1.0,NaN,NaN,13.0,31.0,17.0,NaN,3.0,NaN,NaN,10.0,10.0,31.0,NaN,NaN,NaN,NaN,3.0,23.0,NaN,19.0,NaN,NaN,NaN,NaN,18.0,25.0,NaN,15.0,NaN,20.0,5.0,2.0,1.0,NaN,...,6.0,9.0,20.0,3.0,NaN,6.0,5.0,NaN,6.0,20.0,NaN,NaN,26.0,24.0,NaN,NaN,1.0,NaN,NaN,NaN,28.0,5.0,NaN,NaN,NaN,NaN,NaN,14.0,1.0,1.0,NaN,NaN,3.0,11.0,7.0,18.0,NaN,7.0,NaN,5.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,647.0
세이프란 전용 란셋 니들 26g (100pcs) 1box,6.0,NaN,NaN,NaN,26.0,10.0,NaN,NaN,6.0,NaN,NaN,6.0,NaN,NaN,6.0,10.0,NaN,NaN,NaN,39.0,9.0,NaN,1.0,10.0,6.0,NaN,12.0,35.0,38.0,30.0,6.0,NaN,5.0,1.0,NaN,10.0,NaN,6.0,NaN,6.0,NaN,NaN,NaN,6.0,NaN,NaN,11.0,NaN,6.0,10.0,...,NaN,6.0,NaN,NaN,NaN,40.0,10.0,NaN,NaN,6.0,NaN,NaN,9.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,20.0,NaN,NaN,10.0,NaN,NaN,NaN,NaN,12.0,5.0,1.0,NaN,NaN,6.0,8.0,NaN,NaN,NaN,5.0,NaN,NaN,10.0,15.0,611.0
"동방 일회용 부항컵 1,000개",17.0,13.0,1.0,5.0,26.0,18.0,11.0,7.0,12.0,4.0,NaN,16.0,23.0,17.0,13.0,10.0,2.0,2.0,17.0,21.0,11.0,8.0,10.0,4.0,3.0,2.0,8.0,3.0,10.0,10.0,2.0,NaN,6.0,5.0,12.0,3.0,6.0,1.0,NaN,10.0,8.0,4.0,9.0,3.0,7.0,NaN,7.0,11.0,5.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N